In [1]:
# import datetime
# import json
# import os
# from io import StringIO
# from statistics import mean

# import gpxpy
# import srtm

# from bqplot import Axis, Figure, Lines, LinearScale
# from bqplot.interacts import IndexSelector
# from ipyleaflet import basemaps, FullScreenControl, LayerGroup, Map, MeasureControl, Polyline, Marker, CircleMarker, WidgetControl
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link
import ipywidgets as widgets

In [2]:
# upload the model with its weights
# model = ...

In [3]:
with open('eXamine.png', 'rb') as f:
        image = f.read()
examine_logo = Image(value=image, format='png',layout=Layout(padding='10px'))
display(examine_logo)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01}\x00\x00\x00z\x08\x06\x00\x00\x00S\x0b\x83]\x00\…

In [4]:
tools = ["voila", "ipywidgets"]
logos = []
for tool in tools:
    with open(f'{tool}.png', 'rb') as f:
        image = f.read()
    img = Image(value=image, format='png',layout=Layout(padding='20px'))
    logos.append(img)
HBox([Label(value='Powered by:')] + logos, layout=Layout(flex_flow='row', align_items='center'))

In [6]:
import pickle
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import label_binarize
from tensorflow.keras.layers import Conv2D, add
from PIL import Image
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array,load_img

filename = 'model_weights_430.pickle'
infile = open(filename,'rb')
w_old_unpickled = pickle.load(infile)
infile.close()

img_input = layers.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

#x = residual_module(x, 64)

x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dropout(0.5)(x)

output = layers.Dense(3, activation='softmax')(x)

model = Model(img_input, output)
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['acc'])

model.set_weights(w_old_unpickled)

def predict_img(img):
    # resize the img to match img_input
    img_test = load_img(img, target_size=(150, 150))
    img_test = img_to_array(content)
    img_test = img_test.reshape((1,) + (1, 150, 150, 3))
    res = model.predict(img_test)
    print('benign confidence: {0}%'.format(res[0]))
    print('malignant confidence: {0}%'.format(res[1]))
    print('normal confidence: {0}%'.format(res[2]))

In [46]:
def confidence_bars(res):
    # layout = widgets.Layout(width='auto', height='40px')
    # style = {'description_width: 'initial'}

    normal_percent_confidence = res[2]*100
    malignant_percent_confidence = res[1]*100
    benign_percent_confidence = res[0]*100

    all_confidences = [normal_percent_confidence, malignant_percent_confidence, benign_percent_confidence]

    if np.max(all_confidences) == normal_percent_confidence:
        normal_color = 'green'
    if np.min(all_confidences) == normal_percent_confidence:
        normal_color = 'red'
    if np.max(all_confidences) != normal_percent_confidence and np.min(all_confidences) != normal_percent_confidence: 
        normal_color = 'yellow'

    if np.max(all_confidences) == malignant_percent_confidence:
        malignant_color = 'green'
    if np.min(all_confidences) == malignant_percent_confidence:
        malignant_color = 'red'
    if np.max(all_confidences) != malignant_percent_confidence and np.min(all_confidences) != malignant_percent_confidence:
        malignant_color = 'yellow'

    if np.max(all_confidences) == benign_percent_confidence:
        benign_color = 'green'
    if np.min(all_confidences) == benign_percent_confidence:
        benign_color = 'red'
    if np.max(all_confidences) != benign_percent_confidence and np.min(all_confidences) != benign_percent_confidence:
        benign_color = 'yellow'




    normalc = widgets.IntProgress(
        value=normal_percent_confidence,
        min=0,
        max=100,
        # description='Normal Confidence:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': normal_color},
        orientation='horizontal',
        # style = style
    )
    malignantc = widgets.IntProgress(
        value=malignant_percent_confidence,
        min=0,
        max=100,
        # description='Malignant Confidence:',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': malignant_color},
        orientation='horizontal',
        # style = style
    )
    benignc = widgets.IntProgress(
        value=benign_percent_confidence,
        min=0,
        max=100,
        # description= str(benign_percent_confidence)+'%',
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': benign_color},
        orientation='horizontal',
        # style=style
    )

    # widgets.VBox([normalc, malignantc, benignc])

    display(widgets.VBox([widgets.HBox([widgets.Label('Normal Confidence: ' + str(normal_percent_confidence)+'% '), normalc]),
                widgets.HBox([widgets.Label('Malignant Confidence: '+ str(malignant_percent_confidence)+'% '), malignantc]),
                widgets.HBox([widgets.Label('Benign Confidence: '+str(benign_percent_confidence)+'% '), benignc])]))

In [59]:
def generate_results(r):

    normal_percent = int(r[2]*100)
    benign_percent = int(r[0]*100)
    malignant_percent = int(r[1]*100)

    print("===========================")
    print("||    normal    ||   {0}% ||".format(normal_percent))
    print("===========================")
    print("||    benign    ||   {0}% ||".format(benign_percent))
    print("===========================")
    print("||   malignant  ||   {0}% ||".format(malignant_percent))
    print("===========================")

In [60]:
out = Output()
display(out)

Output()

In [61]:
import numpy as np
import PIL.Image as Image
import io
import base64

uploader = widgets.FileUpload()
display(uploader)

def on_upload_change(change):
    uploaded_file = uploader.value
    #print(uploaded_file)
    key = list(uploaded_file.keys())[0]
    #print(key)
    content = uploaded_file[key]['content']
    display(out)
    out.clear_output()
    with out:
        display(widgets.Image(value=content))
        
        
        img_test=Image.open(io.BytesIO(content))
        #img_test.show()
        #img_test = img_test.convert('1')
        size = (150,150)
        img_test = img_test.resize(size)
        img_test = img_to_array(img_test)
        #print(img_test.shape)
        img_test = img_test.reshape((1,) + img_test.shape)
        res = model.predict(img_test)[0]
        #print(res)
        print()
        generate_results(res)
        print()
        confidence_bars(res)
        
        
uploader.observe(on_upload_change, names='_counter')

FileUpload(value={}, description='Upload')

Output()

In [2]:
layout = widgets.Layout(width='auto', height='40px') #set width and height

button = widgets.Button(
    description='Contact a Medical Professional',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    display='flex',
    flex_flow='column',
    align_items='stretch', 
    layout = layout,
    value = 'https://www.cancer.org/treatment/support-programs-and-services.html'
)        
display(button)

Button(button_style='danger', description='Contact a Medical Professional', icon='check', layout=Layout(height…

In [41]:
# uploaded_file = uploader.value
# content = uploaded_file['benign+(1).png']['content']
# display(widgets.Image(value=content))


In [27]:
# uploader()

In [28]:
# img =  widgets.HTML('<div id="img" style="float:left">'
#                     '<img style="vertical-align:middle" src="http://localhost:8888/notebooks/static/test_image.jpg" alt="">'                       
#                     '</div>')
# img

In [29]:
# import pandas as pd
# import ipywidgets as widgets
# from ipywidgets import Layout

# df = pd.read_csv('SF_Crimes.csv')
# style = {'description_width': 'initial'}

In [30]:
# limit_case = widgets.IntSlider(
#     value=1000,
#     min=100,
#     max=5000,
#     step=1,
#     description='Max Number of Case:',
#     disabled=False,
#     style=style)

# unique_district = df.PdDistrict.unique()

In [31]:
# district = widgets.SelectMultiple(
#     options = unique_district.tolist(),
#     value = ['BAYVIEW', 'NORTHERN'],
#     #rows=10,
#     description='District',
#     disabled=False,
#     layout = Layout(width='50%', height='80px')
# )

In [32]:
# unique_cat = df.Category.unique()

# category = widgets.SelectMultiple(
#     options = unique_cat.tolist(),
#     value = ['VANDALISM', 'ASSAULT', 'ROBBERY'],
#     description='Criminal Case',
#     disabled=False,
#     style=style,
#     layout = Layout(width='50%', height='80px')
# )

In [33]:
# category